In [1]:
import os
import time
import torch
import argparse
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from collections import defaultdict
import torch.multiprocessing as mp
import torch.utils.data
import torch
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import numpy

from sklearn import preprocessing

from models import VAE


device = torch.device('cuda:0')

# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# load the training and test datasets
# train_data = datasets.MNIST(root='data', train=True,
#                                    download=True, transform=transform)
# test_data = datasets.MNIST(root='data', train=False,
#                                   download=True, transform=transform)

# data_loader = DataLoader(
#     dataset=dataset, batch_size=args.batch_size, shuffle=True)
# print(len(train_data))

# dataset = MNIST(
#     root='data', train=True, transform=transforms.ToTensor(),
#     download=True)

slides_3 = np.load('../data/force_torque_sensor/Dataset/3_sides/Data/data.npy')

dataset = slides_3
DATA=dataset
MAX=np.max(slides_3,0)
MIN=np.min(slides_3,0)

print(MAX)
print(MIN)

dataset = slides_3[:,0:10]

print(type(dataset))

# for images in t
# images = images.view(images.size(0), -1)
print(len(dataset[1]))



[8.59982571e-03 8.41197524e-03 1.84235611e-04 8.74299074e-02
 4.72123347e+00 8.83503356e+00 1.77576665e+00 3.23750009e-01
 6.40706637e-01 1.52313336e-01 1.58400000e+03 1.00096209e+01]
[-8.56557263e-03 -8.47084910e-03 -1.83196547e-04 -8.74442646e-02
 -7.90796674e+00 -3.74110003e+00 -3.83027245e+01 -1.48956004e+00
 -1.10945995e+00 -2.19713331e-01  1.00000000e+00  5.38825989e-05]
<class 'numpy.ndarray'>
10


In [2]:
MAX=np.max(slides_3[:,0])
print(MAX)

0.008599825708927977


In [3]:
def feature_normalize(data):
    mu = np.mean(data,axis=0)
    std = np.std(data,axis=0)
    return (data - mu)/std

def un_normalize(normalized_data, input_data):
    mu = np.mean(input_data,axis=0)
    std = np.std(input_data,axis=0)
    return normalized_data*std+mu

In [4]:
# Create training and test dataloaders

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 1024


# valid_size = 0.2

# # obtain training indices that will be used for validation
# num_train = len(dataset)
# indices = list(range(num_train))
# np.random.shuffle(indices)
# split = int(np.floor(valid_size * num_train))
# train_idx, valid_idx = indices[split:], indices[:split]

# # define samplers for obtaining training and validation batches
# train_sampler = SubsetRandomSampler(train_idx)
# valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders
# train_loader = torch.utils.data.DataLoader(train_data,batch_size=batch_size,num_workers=num_workers,sampler=train_sampler)
# valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers,sampler=valid_sampler)
# test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)

# dataset = feature_normalize(dataset)
dataset = preprocessing.minmax_scale(dataset, feature_range=(0,1))

data_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)
train_loader = data_loader
valid_loader = data_loader

# train_loader = torch.utils.data.DataLoader(data_loader,batch_size=batch_size,num_workers=num_workers,sampler=train_sampler)
# valid_loader = torch.utils.data.DataLoader(data_loader, batch_size=batch_size, num_workers=num_workers,sampler=valid_sampler)
# test_loader = torch.utils.data.DataLoader(data_loader, batch_size=batch_size, num_workers=num_workers)

# print(len(train_loader))
# print(len(valid_loader))
# print(len(test_loader))

print(dataset[100])

DATA=dataset
MAX=np.max(DATA,0)
MIN=np.min(DATA,0)

print(MAX)
print(MIN)

[0.00547928 0.0014438  0.44612521 0.00126578 0.60616138 0.30129822
 0.95536093 0.81744986 0.63819255 0.61455272]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


Loss function
--------------------



In [5]:
def loss_fn(recon_x, x, mean, log_var):
#     criterion = nn.BCEWithLogitsLoss(reduction='sum')
#     BCE = criterion(recon_x, x) / len(x)

#     recon_x = torch.clamp(recon_x, 0, 1)
#     x = torch.clamp(x, 0, 1) 
    
    BCE = torch.nn.functional.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())
    # elbo = recon_loss + self.beta * D_kl
    elbo = (BCE + KLD)/ x.size(0)
    return elbo

In [6]:
seed = 0
epochs = 5
latent_size = 2
num_processes = 2
learning_rate = 0.001
print_every = 100
# encoder_layer_sizes=[784, 256]
# decoder_layer_sizes=[256, 784]

vae = VAE(latent_size=latent_size).float().to(device)

# vae = VAE(
#     encoder_layer_sizes=encoder_layer_sizes,
#     latent_size=latent_size,
#     decoder_layer_sizes=decoder_layer_sizes).to(device)

use_cuda = torch.cuda.is_available
if use_cuda:
    vae.to(device)

logs = defaultdict(list)
    
epochs = epochs
for epoch in range(epochs):
    lr_havel = learning_rate
    optimizer = torch.optim.Adam(vae.parameters(), lr=lr_havel)

    for iteration, d in enumerate(train_loader):
        if use_cuda:
            input_data = d.to(device).float()
        else:
            input_data = d.float()
        optimizer.zero_grad()
        recon_x, mean, log_var, z = vae(input_data)
        elbo = loss_fn(recon_x, input_data, mean, log_var)
        loss = elbo
        loss.backward()
        optimizer.step()

        logs['loss'].append(loss.item())

        if iteration % print_every == 0 or iteration == len(train_loader)-1:
            valid_correct = 0
            valid_total = 0


            for valid in valid_loader:
                if use_cuda:
                    input_valid = valid.to(device).float()
                else:
                    input_valid = valid.float()
                valid_recon_x, valid_mean, valid_log_var, valid_z = vae(input_valid)
                valid_elbo = loss_fn(valid_recon_x, input_valid, valid_mean, valid_log_var)
                criterion = nn.MSELoss()
                valid_loss = criterion(valid_recon_x, input_valid)
                logs['valid_elbo'].append(valid_elbo.item())
                logs['valid_loss'].append(valid_loss.item())
                valid_total +=1
                if valid_loss < 0.05:
                    valid_correct += 1
            valid_acc = valid_correct / valid_total  
                
            print("Epoch {:02d}/{:02d} Batch {:04d}/{:d} |Train_ELBO {:5f} |Valid_ELBO{:5f} |Valid_loss{:5f} |Valid_accuary{:5f} |Leaarning_rate{:5f}".format(
                epoch, epochs, iteration, len(train_loader)-1, loss.item(), valid_elbo.item(), valid_loss.item(), valid_acc, lr_havel))

    lr_havel = lr_havel*0.9
    torch.save(vae, 'vae_ft_inary_cross_entropy_sklearn_preprocess_0_255.pkl')  

Epoch 00/05 Batch 0000/1548 |Train_ELBO 7.314663 |Valid_ELBO7.262753 |Valid_loss0.062135 |Valid_accuary0.000000 |Leaarning_rate0.001000
Epoch 00/05 Batch 0100/1548 |Train_ELBO 6.650987 |Valid_ELBO6.644906 |Valid_loss0.042994 |Valid_accuary1.000000 |Leaarning_rate0.001000
Epoch 00/05 Batch 0200/1548 |Train_ELBO 6.557600 |Valid_ELBO6.572185 |Valid_loss0.040000 |Valid_accuary1.000000 |Leaarning_rate0.001000
Epoch 00/05 Batch 0300/1548 |Train_ELBO 6.561653 |Valid_ELBO6.563843 |Valid_loss0.040581 |Valid_accuary1.000000 |Leaarning_rate0.001000
Epoch 00/05 Batch 0400/1548 |Train_ELBO 6.554377 |Valid_ELBO6.564924 |Valid_loss0.038860 |Valid_accuary1.000000 |Leaarning_rate0.001000
Epoch 00/05 Batch 0500/1548 |Train_ELBO 6.557536 |Valid_ELBO6.553649 |Valid_loss0.040489 |Valid_accuary1.000000 |Leaarning_rate0.001000
Epoch 00/05 Batch 0600/1548 |Train_ELBO 6.564477 |Valid_ELBO6.549419 |Valid_loss0.043391 |Valid_accuary1.000000 |Leaarning_rate0.001000
Epoch 00/05 Batch 0700/1548 |Train_ELBO 6.564385

Epoch 03/05 Batch 1000/1548 |Train_ELBO 6.565870 |Valid_ELBO6.571492 |Valid_loss0.041010 |Valid_accuary1.000000 |Leaarning_rate0.001000
Epoch 03/05 Batch 1100/1548 |Train_ELBO 6.578108 |Valid_ELBO6.557882 |Valid_loss0.038810 |Valid_accuary1.000000 |Leaarning_rate0.001000
Epoch 03/05 Batch 1200/1548 |Train_ELBO 6.568311 |Valid_ELBO6.574726 |Valid_loss0.039270 |Valid_accuary1.000000 |Leaarning_rate0.001000
Epoch 03/05 Batch 1300/1548 |Train_ELBO 6.574415 |Valid_ELBO6.554903 |Valid_loss0.040979 |Valid_accuary1.000000 |Leaarning_rate0.001000
Epoch 03/05 Batch 1400/1548 |Train_ELBO 6.571330 |Valid_ELBO6.567421 |Valid_loss0.040950 |Valid_accuary1.000000 |Leaarning_rate0.001000
Epoch 03/05 Batch 1500/1548 |Train_ELBO 6.575253 |Valid_ELBO6.553588 |Valid_loss0.041130 |Valid_accuary1.000000 |Leaarning_rate0.001000
Epoch 03/05 Batch 1548/1548 |Train_ELBO 6.579605 |Valid_ELBO6.539072 |Valid_loss0.040940 |Valid_accuary1.000000 |Leaarning_rate0.001000
Epoch 04/05 Batch 0000/1548 |Train_ELBO 6.555912

In [7]:
torch.save(vae, 'vae_10signals.pkl')



[-1.58900694 -1.57128417 -0.3725314  -1.58027392  0.13725066  0.31933986
  1.8955267  -0.23969703  0.01458174 -0.11522518]


## Testing>>>>>>

In [ ]:

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 16


# dataset = feature_normalize(dataset)
dataset = preprocessing.minmax_scale(dataset, feature_range=(0,1))


test_loader = DataLoader(dataset=dataset, batch_size=batch_size)

In [30]:
recon_signals=[]
latent_space=[]

import torch.utils.data

vae=vae.to(device)
test_loader = DataLoader(dataset=dataset, batch_size=2)

recon_signals=[]
latent_space=[]

import torch.utils.data

vae=vae.to(device)

for iteration, test in enumerate(test_loader):
    test = test.to(device).float()
    test_recon_x, test_mean, test_log_var, test_z = vae(test)
    test_recon_x = test_recon_x.view(batch_size, len(test_recon_x[0]))
    test_recon_x=test_recon_x.cpu().detach().numpy()
    latent_space=test_z.cpu().detach().numpy()

    recon_signals.extend(test_recon_x)
#     latent_space.extend(latent_space)
    
    if iteration ==10000:
        break
    
recon_signals = np.array(recon_signals)
latent_space = np.array(latent_space)

RuntimeError: CUDA error: unspecified launch failure

In [16]:
print(type(recon_signals[0]))
print(recon_signals[0])

print(len(recon_signals))
print(recon_signals)

<class 'numpy.ndarray'>
[0.48990786 0.5268015  0.4728889  0.47508365 0.5395386  0.29535234
 0.3683021  0.78276724 0.58985096 0.61199296]
1001
[array([0.48990786, 0.5268015 , 0.4728889 , 0.47508365, 0.5395386 ,
       0.29535234, 0.3683021 , 0.78276724, 0.58985096, 0.61199296],
      dtype=float32), array([0.49564868, 0.513131  , 0.47475687, 0.48778754, 0.5441126 ,
       0.29167354, 0.37800756, 0.79066736, 0.58829254, 0.61567336],
      dtype=float32), array([0.5057561 , 0.5178004 , 0.475026  , 0.48305592, 0.54551893,
       0.28386366, 0.38212016, 0.787678  , 0.59405375, 0.626582  ],
      dtype=float32), array([0.50203496, 0.48884025, 0.4787087 , 0.49811575, 0.54754263,
       0.292998  , 0.39067253, 0.79298335, 0.5894255 , 0.6165135 ],
      dtype=float32), array([0.49656972, 0.511831  , 0.46259907, 0.4889395 , 0.547284  ,
       0.2711612 , 0.36853787, 0.81357944, 0.5924854 , 0.6240586 ],
      dtype=float32), array([0.50204206, 0.4888484 , 0.47870913, 0.49810675, 0.54754335,
     

In [29]:
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt

cmap=plt.cm.get_cmap(plt.cm.viridis,143)


# plt.rcParams['font.size'] = 11.
# plt.rcParams['font.family'] = 'Comic Sans MS'
# plt.rcParams['axes.labelsize'] = 15.
# plt.rcParams['xtick.labelsize'] = 10.
# plt.rcParams['ytick.labelsize'] = 10.

plt.figure(figsize=(15,10))



sides = generate_signals
print(len(sides))

slip = 3000

# Peg_Position_x = sides[:,0]
Peg_Position_x = sides[:,0][:slip]
# Peg_Position_x = sides[:,0][0:10:slip]

# Peg_Position_y = sides[:,1]
Peg_Position_y = sides[:,1][:slip]
# Peg_Position_y = sides[:,1][0:10:slip]

# Peg_Position_z = sides[:,2]
Peg_Position_z = sides[:,2][:slip]
# Peg_Position_z = sides[:,2][0:10:slip]

# Angle = sides[:,3]
Angle = sides[:,3][:slip]
# Angle = sides[:,3][0:10:slip]

# Force_x = sides[:,4]
Force_x = sides[:,4][:slip]
# Force_x = sides[:,4][0:10:slip]

# Force_y = sides[:,5]
Force_y = sides[:,5][:slip]
# Force_y = sides[:,5][0:10:slip]

# Force_z = sides[:,6]
Force_z = sides[:,6][:slip]
# Force_z = sides[:,6][0:10:slip]

# Moment_x = sides[:,7]
Moment_x = sides[:,7][:slip]
# Moment_x = sides[:,7][0:10:slip]

# Moment_y = sides[:,8]
Moment_y = sides[:,8][:slip]
# Moment_y = sides[:,8][0:10:slip]

# Moment_z = sides[:,9]
Moment_z = sides[:,9][:slip]
# Moment_z = sides[:,9][0:10:slip]

x = np.linspace(0,len(Moment_z),len(Moment_z))

plt.plot(x, Force_x, marker='o', linestyle='-', markersize=2, linewidth=1, label='Force_x')
plt.plot(x, Force_y, marker='o', linestyle='-', markersize=2, linewidth=1, label='Force_y')
plt.plot(x, Force_z, marker='o', linestyle='-', markersize=2, linewidth=1, label='Force_z')

plt.plot(x, Moment_x, marker='o', linestyle='-', markersize=2, linewidth=1, label='Moment_x')
plt.plot(x, Moment_y, marker='o', linestyle='-', markersize=2, linewidth=1, label='Moment_y')
plt.plot(x, Moment_z, marker='o', linestyle='-', markersize=2, linewidth=1, label='Moment_z')

plt.plot(x, Peg_Position_x, marker='o', linestyle='-', markersize=2, linewidth=1, label='Peg_Position_x')
plt.plot(x, Peg_Position_y, marker='o', linestyle='-', markersize=2, linewidth=1, label='Peg_Position_y')
plt.plot(x, Peg_Position_z, marker='o', linestyle='-', markersize=2, linewidth=1, label='Peg_Position_z')

plt.plot(x, Angle, marker='o', linestyle='-', markersize=2, linewidth=1, label='Angle')
# plt.plot(x, Counter, marker='o', linestyle='-', markersize=2, linewidth=1, label='Counter')


plt.xlabel('Times')
plt.ylabel('Signals')
plt.title('Force Torque Signals(3 Siles)')
plt.legend(loc='lower right')
plt.show()

TypeError: list indices must be integers or slices, not tuple

In [25]:
print(recon_signals)

[array([-0.00124059, -0.00152473, -0.00243168, -0.00134705,  0.00027437,
       -0.00856557, -0.00521658,  0.00859982,  0.00140854,  0.00250705],
      dtype=float32), array([-0.00126645, -0.00154591, -0.00243793, -0.00137116,  0.00022357,
       -0.00847085, -0.00517699,  0.00841198,  0.00133907,  0.0024195 ],
      dtype=float32), array([-2.64024184e-05, -3.24844077e-05, -5.18981906e-05, -2.86812665e-05,
        6.02586306e-06, -1.83196549e-04, -1.11510046e-04,  1.84235629e-04,
        3.03032266e-05,  5.38173881e-05], dtype=float32), array([-0.0128203 , -0.01571494, -0.02495466, -0.01390489,  0.00261348,
       -0.08744427, -0.05332609,  0.08742992,  0.01416795,  0.02535919],
      dtype=float32), array([-2.5187144 , -2.7277617 , -3.395043  , -2.597042  , -1.4041053 ,
       -7.9079666 , -5.4439926 ,  4.7212343 , -0.56965554,  0.23856175],
      dtype=float32), array([ 1.6255071,  1.4173383,  0.752861 ,  1.5475087,  2.7354329,
       -3.7411   , -1.2874794,  8.835034 ,  3.5663762,  

In [ ]:
# batch_size = 10
z = torch.randn([batch_size, 2]).to(device)
print(z)

recon_x = vae.decoder(z)

print(recon_x)